# Covid-19 Step By Step Data Analysis

A exercise for self practising purpose. Thanks to the contributors below for providing the tutorial and explanation.
Credit to
- [COVID-19 data processing with Pandas DataFrame](https://towardsdatascience.com/covid-19-data-processing-58aaa3663f6) by B.Chen
- [Automated ETL for LIVE Tableau Public Visualizations](https://towardsdatascience.com/automated-etl-for-live-tableau-public-visualizations-54f2b8652224) by Eklavya Saxena
- [COVID-19 Forecasting Challenges (Week 2) Data Prep](https://www.kaggle.com/benhamner/covid-19-forecasting-challenges-week-2-data-prep)

## 0. Preparation

The plotly version in Kaggle is 4.8.1, downgrade to 3.10.0 to use `ploty.ploty`

In [1]:
!pip install plotly==3.10.0

     |████████████████████████████████| 41.5 MB 159 kB/s 
ERROR: plotly-express 0.4.1 has requirement plotly>=4.1.0, but you'll have plotly 3.10.0 which is incompatible.
ERROR: cufflinks 0.17.3 has requirement plotly>=4.1.1, but you'll have plotly 3.10.0 which is incompatible.
ERROR: catalyst 20.6 has requirement plotly>=4.1.0, but you'll have plotly 3.10.0 which is incompatible.
  Attempting uninstall: plotly
    Found existing installation: plotly 4.8.1
    Uninstalling plotly-4.8.1:
      Successfully uninstalled plotly-4.8.1


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualization library
import matplotlib.pyplot as plt # visualization library
import plotly.plotly as py # visualization library
from plotly.offline import init_notebook_mode, iplot # plotly offline mode
init_notebook_mode(connected=True) 
import plotly.graph_objs as go # plotly graphical object
from datetime import date, datetime, timedelta


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
print(os.listdir("../input"))
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# import warnings library
import warnings        
# ignore filters
warnings.filterwarnings("ignore") # if there is a warning after some codes, this will avoid us to see them.
plt.style.use('ggplot') # style of plots. ggplot is one of the most used style, I also like it.
# Any results you write to the current directory are saved as output.

['time_series_covid19_recovered_global.csv', 'time_series_covid19_deaths_global.csv', 'time_series_covid19_confirmed_global.csv']
/kaggle/input/time_series_covid19_recovered_global.csv
/kaggle/input/time_series_covid19_deaths_global.csv
/kaggle/input/time_series_covid19_confirmed_global.csv


## 1. Data Processing
Always observe the raw data format and have some insight of how to manipulating data.

Data is downloaded from
[Johns Hopkins CSSE public Github page](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series)

In [3]:
confirmed = pd.read_csv("../input/time_series_covid19_confirmed_global.csv")
deaths = pd.read_csv("../input/time_series_covid19_deaths_global.csv")
recovered = pd.read_csv("../input/time_series_covid19_recovered_global.csv")

In [4]:
confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,22142,22890,23546,24102,24766,25527,26310,26874,27532,27878
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1341,1385,1416,1464,1521,1590,1672,1722,1788,1838
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,10484,10589,10698,10810,10919,11031,11147,11268,11385,11504
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,852,852,853,853,853,853,854,854,855,855
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,113,118,130,138,140,142,148,155,166,172


### Observation:
- There are 3 time series data, *confirmed*, *deaths* and *recovered* that are stored in different CSV files. This would be an issue when trying to plot in the same chart in data visualization.
- Dates are in columns, it's not possible to simply use Dates as one of the axis. (Have to call all date columns to show all dates at once)
- *confirmed*, *deaths* and *recovered* are not matched due to missing / mismatch, and some countries do not have the recovered figures in Province/State level
- Cruise ships are mismatch in Province/State column

In [5]:
confirmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Columns: 154 entries, Province/State to 6/19/20
dtypes: float64(2), int64(150), object(2)
memory usage: 320.2+ KB


All tables have the same columns `Province_State`, `Country_Region`, `Lat`, `Long`, following by the dates

In [6]:
print(confirmed.columns)
print(deaths.columns)
print(recovered.columns)

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '6/10/20', '6/11/20', '6/12/20', '6/13/20', '6/14/20', '6/15/20',
       '6/16/20', '6/17/20', '6/18/20', '6/19/20'],
      dtype='object', length=154)
Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '6/10/20', '6/11/20', '6/12/20', '6/13/20', '6/14/20', '6/15/20',
       '6/16/20', '6/17/20', '6/18/20', '6/19/20'],
      dtype='object', length=154)
Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '6/10/20', '6/11/20', '6/12/20', '6/13/20', '6/14/20', '6/15/20',
       '6/16/20', '6/17/20', '6/18/20', '6/19/20'],
      dtype='object', length=154)


## 2. Data Formatting
- Format Date from `m/dd/yy` to `yyyy-mm-dd`
- Rename 'Province/State' as `Province_State` and 'Country/Region' as `Country_Region`

In [7]:
confirmed.columns = list(confirmed.columns[:4]) + [datetime.strptime(d, "%m/%d/%y").date().strftime("%Y-%m-%d") for d in confirmed.columns[4:]]
deaths.columns    = list(deaths.columns[:4])    + [datetime.strptime(d, "%m/%d/%y").date().strftime("%Y-%m-%d") for d in deaths.columns[4:]]
recovered.columns = list(recovered.columns[:4]) + [datetime.strptime(d, "%m/%d/%y").date().strftime("%Y-%m-%d") for d in recovered.columns[4:]]

In [8]:
confirmed.rename(columns={"Province/State": "Province_State", "Country/Region": "Country_Region"}, inplace=True)
deaths.rename(columns={"Province/State": "Province_State", "Country/Region": "Country_Region"}, inplace=True)
recovered.rename(columns={"Province/State": "Province_State", "Country/Region": "Country_Region"}, inplace=True)

In [9]:
confirmed.head()

,Province_State,Country_Region,Lat,Long,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-19
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,22142,22890,23546,24102,24766,25527,26310,26874,27532,27878
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1341,1385,1416,1464,1521,1590,1672,1722,1788,1838
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,10484,10589,10698,10810,10919,11031,11147,11268,11385,11504
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,852,852,853,853,853,853,854,854,855,855
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,113,118,130,138,140,142,148,155,166,172


### Merging tables of Confirmed, Deaths and Recovered

use `melt()` to unpivot DataFrames from wide format to long format
- Identifier variables: `Province_State`, `Country_Region`, `Lat`, `Long`
- Unpivor date columns (`columns[4:]`) with variable column `Date` and value column `Confirmed`

In [10]:
dates = confirmed.columns[4:]

confirmed_long = confirmed.melt(
    id_vars=['Province_State', 'Country_Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Confirmed'
)
deaths_long = deaths.melt(
    id_vars=['Province_State', 'Country_Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Deaths'
)
recovered_long = recovered.melt(
    id_vars=['Province_State', 'Country_Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Recovered'
)

This would return a long format table that
- ordered by `Date` and `Country_Region` (raw data was ordered by these columns in ASC order)

In [11]:
confirmed_long

,Province_State,Country_Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.000000,65.000000,2020-01-22,0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0
...,...,...,...,...,...,...
39895,NaN,Sao Tome and Principe,0.186360,6.613081,2020-06-19,693
39896,NaN,Yemen,15.552727,48.516388,2020-06-19,919
39897,NaN,Comoros,-11.645500,43.333300,2020-06-19,210
39898,NaN,Tajikistan,38.861034,71.276093,2020-06-19,5338


In [12]:
(confirmed_long['Country_Region'] == "Canada").value_counts()

False    37800
True      2100
Name: Country_Region, dtype: int64

In [13]:
recovered_long[recovered_long['Country_Region']=='Canada']

,Province_State,Country_Region,Lat,Long,Date,Recovered
36,NaN,Canada,56.1304,-106.3468,2020-01-22,0
289,NaN,Canada,56.1304,-106.3468,2020-01-23,0
542,NaN,Canada,56.1304,-106.3468,2020-01-24,0
795,NaN,Canada,56.1304,-106.3468,2020-01-25,0
1048,NaN,Canada,56.1304,-106.3468,2020-01-26,0
...,...,...,...,...,...,...
36721,NaN,Canada,56.1304,-106.3468,2020-06-15,61466
36974,NaN,Canada,56.1304,-106.3468,2020-06-16,61899
37227,NaN,Canada,56.1304,-106.3468,2020-06-17,63280
37480,NaN,Canada,56.1304,-106.3468,2020-06-18,63782


Remove recovered data for Canada due to mismatch issue (Canada recovered data is counted by Country-wise rather than Province/State-wise).

In [14]:
recovered_long = recovered_long[recovered_long['Country_Region']!='Canada']

In [15]:
# Check if there is Cana
(recovered_long['Province_State']=='Canada').value_counts()

False    37800
Name: Province_State, dtype: int64

Use `merge()` to merge 3 DataFrame one after another

Merging 1: Use confirmed table as referenced table, left joined by the `deaths_long` table and result as `full_table`

Merging 2: `full_table` left joined by the `recovered_long`

In [16]:
# Merging 1: create full_table using confirmed_long and left joined by deaths_long
full_table = confirmed_long.merge(
    right = deaths_long,
    how = 'left',
    on = ['Province_State', 'Country_Region', 'Date', 'Lat', 'Long']
)

# Merging 2: merge full_table with recovered_long
full_table = full_table.merge(
    right = recovered_long,
    how = 'left',
    on = ['Province_State', 'Country_Region', 'Date', 'Lat', 'Long']
)

Now we have a full table with `confirmed`, `deaths` and `recovered`

In [17]:
full_table

,Province_State,Country_Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.000000,65.000000,2020-01-22,0,0,0.0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0.0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0.0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0.0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0.0
...,...,...,...,...,...,...,...,...
39895,NaN,Sao Tome and Principe,0.186360,6.613081,2020-06-19,693,12,199.0
39896,NaN,Yemen,15.552727,48.516388,2020-06-19,919,251,288.0
39897,NaN,Comoros,-11.645500,43.333300,2020-06-19,210,5,129.0
39898,NaN,Tajikistan,38.861034,71.276093,2020-06-19,5338,51,3830.0


## 3. Data Cleansing

1. ~~Convert `Date` from `string` to `datetime`~~ (Done in Data Formatting)
2. Replace missing value `Nan`
3. Coronavirus cases reported from 3 cruise ships are mismatched

In [18]:
# Date is in datetime format
full_table.dtypes

Province_State     object
Country_Region     object
Lat               float64
Long              float64
Date               object
Confirmed           int64
Deaths              int64
Recovered         float64
dtype: object

In [19]:
# Convert Date from String to DateTime
#full_table['Date'] = pd.to_datetime(full_table['Date'])
#full_table.dtypes

Detect missing values `NaN` by running `df.isna().sum()`

There is a lot `NaN` in Province_State, it makes sense as some countries report recovered cases in country level

We will replace the `NaN` with `0`

In [20]:
full_table.isna().sum()

Province_State    27750
Country_Region        0
Lat                   0
Long                  0
Date                  0
Confirmed             0
Deaths                0
Recovered          2550
dtype: int64

In [21]:
full_table['Recovered'] = full_table['Recovered'].fillna(0)
full_table[full_table['Recovered']==0]

,Province_State,Country_Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.000000,65.000000,2020-01-22,0,0,0.0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0.0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0.0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0.0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0.0
...,...,...,...,...,...,...,...,...
39869,NaN,Syria,34.802075,38.996815,2020-06-19,187,7,0.0
39870,NaN,Timor-Leste,-8.874217,125.727539,2020-06-19,24,0,0.0
39878,Northwest Territories,Canada,64.825500,-124.845700,2020-06-19,5,0,0.0
39879,Yukon,Canada,64.282300,-135.000000,2020-06-19,11,0,0.0


The `recovered` columns are now filled with 0 but when you check the Dtype it's in `float64`

Let's convert back to `int` to keep data consistency

In [22]:
full_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39900 entries, 0 to 39899
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province_State  12150 non-null  object 
 1   Country_Region  39900 non-null  object 
 2   Lat             39900 non-null  float64
 3   Long            39900 non-null  float64
 4   Date            39900 non-null  object 
 5   Confirmed       39900 non-null  int64  
 6   Deaths          39900 non-null  int64  
 7   Recovered       39900 non-null  float64
dtypes: float64(3), int64(2), object(3)
memory usage: 2.7+ MB


In [23]:
full_table['Recovered'] = full_table['Recovered'].astype('int')
full_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39900 entries, 0 to 39899
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province_State  12150 non-null  object 
 1   Country_Region  39900 non-null  object 
 2   Lat             39900 non-null  float64
 3   Long            39900 non-null  float64
 4   Date            39900 non-null  object 
 5   Confirmed       39900 non-null  int64  
 6   Deaths          39900 non-null  int64  
 7   Recovered       39900 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 2.7+ MB


### Cruises Ship Data

The table included data of 3 cruise ships: **Grand Princess, Diamond Princess and MS Zaandam**
However, cruise data are mapped to `Province_State` and `Country_Region` and some of them are mismatched to `Canada` as Country

In [24]:
lst = ['Grand Princess', 'Diamond Princess', 'MS Zaandam']
lst = '|'.join(lst)
full_table.loc[full_table['Province_State'].str.contains(lst, na=False) | full_table['Country_Region'].str.contains(lst, na=False)]

,Province_State,Country_Region,Lat,Long,Date,Confirmed,Deaths,Recovered
37,Grand Princess,Canada,37.6489,-122.6655,2020-01-22,0,0,0
88,NaN,Diamond Princess,0.0000,0.0000,2020-01-22,0,0,0
231,Diamond Princess,Canada,0.0000,0.0000,2020-01-22,0,0,0
251,NaN,MS Zaandam,0.0000,0.0000,2020-01-22,0,0,0
303,Grand Princess,Canada,37.6489,-122.6655,2020-01-23,0,0,0
...,...,...,...,...,...,...,...,...
39619,NaN,MS Zaandam,0.0000,0.0000,2020-06-18,9,2,0
39671,Grand Princess,Canada,37.6489,-122.6655,2020-06-19,13,0,0
39722,NaN,Diamond Princess,0.0000,0.0000,2020-06-19,712,13,651
39865,Diamond Princess,Canada,0.0000,0.0000,2020-06-19,0,1,0


We will remove the cruise data by extracting them out of the `full_table`

In [25]:
ship_rows = full_table['Province_State'].str.contains('Grand Princess') | full_table['Province_State'].str.contains('Diamond Princess') | full_table['Country_Region'].str.contains('Diamond Princess') | full_table['Country_Region'].str.contains('MS Zaandam')
full_ship = full_table[ship_rows]
full_ship

,Province_State,Country_Region,Lat,Long,Date,Confirmed,Deaths,Recovered
37,Grand Princess,Canada,37.6489,-122.6655,2020-01-22,0,0,0
88,NaN,Diamond Princess,0.0000,0.0000,2020-01-22,0,0,0
231,Diamond Princess,Canada,0.0000,0.0000,2020-01-22,0,0,0
251,NaN,MS Zaandam,0.0000,0.0000,2020-01-22,0,0,0
303,Grand Princess,Canada,37.6489,-122.6655,2020-01-23,0,0,0
...,...,...,...,...,...,...,...,...
39619,NaN,MS Zaandam,0.0000,0.0000,2020-06-18,9,2,0
39671,Grand Princess,Canada,37.6489,-122.6655,2020-06-19,13,0,0
39722,NaN,Diamond Princess,0.0000,0.0000,2020-06-19,712,13,651
39865,Diamond Princess,Canada,0.0000,0.0000,2020-06-19,0,1,0


Exclude the ship data with `~(ship_rows)`

*Q: What is the tilde (~) sign?*

Ans: Boolean operators
You can negate boolean expressions with the word not or the ~ operator.

```
In [256]: df.query('~bools')
Out[256]: 
          a         b         c  bools
2  0.697753  0.212799  0.329209  False
7  0.275396  0.691034  0.826619  False
8  0.190649  0.558748  0.262467  False

In [257]: df.query('not bools')
Out[257]: 
          a         b         c  bools
2  0.697753  0.212799  0.329209  False
7  0.275396  0.691034  0.826619  False
8  0.190649  0.558748  0.262467  False
```

In [26]:
full_table = full_table[~(ship_rows)]
full_table.loc[full_table['Province_State'].str.contains(lst, na=False) | full_table['Country_Region'].str.contains(lst, na=False)]

,Province_State,Country_Region,Lat,Long,Date,Confirmed,Deaths,Recovered


## 4. Data Aggregation

Create a column of `Active` representing the active covid cases based on,

`Active = Confirmed - Deaths - Recovered`

In [27]:
# Active Case = confirmed - deaths - recovered
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']

### Aggregated by Countries
- `sum()` is to get the total count of `Confirmed`, `Deaths`, `Recovered`, `Active` for the given `Date` and `Country_Region`.
- `reset_index()` reset the index and use the default one, which is `Date` and `Country_Region`.

In [28]:
full_grouped = full_table.groupby(['Date', 'Country_Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
full_grouped

,Date,Country_Region,Confirmed,Deaths,Recovered,Active
0,2020-01-22,Afghanistan,0,0,0,0
1,2020-01-22,Albania,0,0,0,0
2,2020-01-22,Algeria,0,0,0,0
3,2020-01-22,Andorra,0,0,0,0
4,2020-01-22,Angola,0,0,0,0
...,...,...,...,...,...,...
27895,2020-06-19,West Bank and Gaza,675,3,437,235
27896,2020-06-19,Western Sahara,9,1,8,0
27897,2020-06-19,Yemen,919,251,288,380
27898,2020-06-19,Zambia,1430,11,1194,225


### New cases, New deaths, New recovered
- To get the latest day data by deducting the corresponding accumulative data on previous day

Let't go through the code below step by step
```
temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']
temp = temp.sum().diff().reset_index()
mask = temp['Country/Region'] != temp['Country/Region'].shift(1)
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan

# renaming columns
temp.columns = ['Country/Region', 'Date', 'New cases', 'New deaths', 'New recovered']

# merging new values
full_grouped = pd.merge(full_grouped, temp, on=['Country/Region', 'Date'])

# filling na with 0
full_grouped = full_grouped.fillna(0)

# fixing data types
cols = ['New cases', 'New deaths', 'New recovered']
full_grouped[cols] = full_grouped[cols].astype('int')

# 
full_grouped['New cases'] = full_grouped['New cases'].apply(lambda x: 0 if x<0 else x)
```

In [29]:
# new cases 
temp = full_grouped.groupby(['Country_Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']
temp.head()

,Confirmed,Deaths,Recovered
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
27895,675,3,437
27896,9,1,8
27897,919,251,288
27898,1430,11,1194


In [30]:
temp = temp.sum().diff().reset_index()
temp

,Country_Region,Date,Confirmed,Deaths,Recovered
0,Afghanistan,2020-01-22,NaN,NaN,NaN
1,Afghanistan,2020-01-23,0.0,0.0,0.0
2,Afghanistan,2020-01-24,0.0,0.0,0.0
3,Afghanistan,2020-01-25,0.0,0.0,0.0
4,Afghanistan,2020-01-26,0.0,0.0,0.0
...,...,...,...,...,...
27895,Zimbabwe,2020-06-15,4.0,0.0,0.0
27896,Zimbabwe,2020-06-16,4.0,0.0,8.0
27897,Zimbabwe,2020-06-17,10.0,0.0,1.0
27898,Zimbabwe,2020-06-18,62.0,0.0,0.0


In [31]:
mask = temp['Country_Region'] != temp['Country_Region'].shift(1)
mask

0         True
1        False
2        False
3        False
4        False
         ...  
27895    False
27896    False
27897    False
27898    False
27899    False
Name: Country_Region, Length: 27900, dtype: bool

In [32]:
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan
temp.loc[mask, 'Confirmed']

0       NaN
150     NaN
300     NaN
450     NaN
600     NaN
         ..
27150   NaN
27300   NaN
27450   NaN
27600   NaN
27750   NaN
Name: Confirmed, Length: 186, dtype: float64

In [33]:
# renaming columns
temp.columns = ['Country_Region', 'Date', 'New cases', 'New deaths', 'New recovered']
# merging new values
full_grouped = pd.merge(full_grouped, temp, on=['Country_Region', 'Date'])
# filling na with 0
full_grouped = full_grouped.fillna(0)
# fixing data types
cols = ['New cases', 'New deaths', 'New recovered']
full_grouped[cols] = full_grouped[cols].astype('int')
# 
full_grouped['New cases'] = full_grouped['New cases'].apply(lambda x: 0 if x<0 else x)

In [34]:
full_grouped

,Date,Country_Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
0,2020-01-22,Afghanistan,0,0,0,0,0,0,0
1,2020-01-22,Albania,0,0,0,0,0,0,0
2,2020-01-22,Algeria,0,0,0,0,0,0,0
3,2020-01-22,Andorra,0,0,0,0,0,0,0
4,2020-01-22,Angola,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
27895,2020-06-19,West Bank and Gaza,675,3,437,235,75,0,22
27896,2020-06-19,Western Sahara,9,1,8,0,0,0,0
27897,2020-06-19,Yemen,919,251,288,380,10,3,15
27898,2020-06-19,Zambia,1430,11,1194,225,14,0,50


In [35]:
full_grouped.to_csv('COVID-19-time-series-clean-complete.csv')

## 5. Data Exploration

In [36]:
import pandas as pd
import altair as alt
full_grouped = pd.read_csv('COVID-19-time-series-clean-complete.csv', parse_dates=['Date'])
sg = full_grouped[full_grouped['Country_Region'] == 'Singapore']

In [37]:
base = alt.Chart(sg).mark_bar().encode(
    x='monthdate(Date):O',
).properties(
    width=500
)

In [38]:
red = alt.value('#f54242')
base.encode(y='Confirmed').properties(title='Total confirmed') | base.encode(y='Deaths', color=red).properties(title='Total deaths')

alt.HConcatChart(...)

In [39]:
red = alt.value('#f54242')
base.encode(y='New cases').properties(title='Daily new cases') | base.encode(y='New deaths', color=red).properties(title='Daily new deaths')

alt.HConcatChart(...)

## Showing the coronavirus spreads

In [40]:
import pandas as pd
import altair as alt
full_grouped = pd.read_csv('COVID-19-time-series-clean-complete.csv', parse_dates=['Date'])
countries = ['US', 'Italy', 'China', 'Spain', 'Germany', 'France', 'Iran', 'United Kingdom', 'Switzerland','Singapore']
selected_countries = full_grouped[full_grouped['Country_Region'].isin(countries)]
selected_countries

,Unnamed: 0,Date,Country_Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
36,36,2020-01-22,China,548,17,28,503,0,0,0
61,61,2020-01-22,France,0,0,0,0,0,0,0
65,65,2020-01-22,Germany,0,0,0,0,0,0,0
80,80,2020-01-22,Iran,0,0,0,0,0,0,0
84,84,2020-01-22,Italy,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
27864,27864,2020-06-19,Singapore,41615,26,33459,8130,142,0,747
27870,27870,2020-06-19,Spain,245575,28315,150376,66884,307,1179,0
27875,27875,2020-06-19,Switzerland,31235,1956,28900,379,35,0,0
27886,27886,2020-06-19,US,2220961,119112,606715,1495134,29909,678,7600


In [41]:
alt.Chart(selected_countries).mark_circle().encode(
    x='monthdate(Date):O',
    y='Country_Region',
    color='Country_Region',
    size=alt.Size('New cases:Q',
        scale=alt.Scale(range=[0, 1000]),
        legend=alt.Legend(title='Daily new cases')
    ) 
).properties(
    width=1200,
    height=500
)

alt.Chart(...)